# Data collection

In [ ]:
# Import libraries for general plotting
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

#library for dataset
from ucimlrepo import fetch_ucirepo  

# Download dataset from UCI ML repository
cdc_diabetes_health_indicators = fetch_ucirepo(id=891)

# Divide database in two pandas dataframes 
X_base = cdc_diabetes_health_indicators.data.features
y_base = cdc_diabetes_health_indicators.data.targets


# VERIFY NO MISSING VALUES
print(X_base.info())
print(y_base.info())

In [ ]:
#Check the type of values in each feature
for column in X_base.columns:
    print(f"Column: {column}")
    print(X_base[column].unique())
    print("-" * 40)

print(f"Target Column: Diabetes_binary")
print(y_base['Diabetes_binary'].unique())

In [ ]:
# Count number of people with/without diabetes
conteo = y_base['Diabetes_binary'].value_counts()

print(conteo)


### Undersampling to balance diabetic and non-diabetic population

In [ ]:
from sklearn.utils import resample

# Combine X and y for consistent resampling
data = X_base.copy()
data['Diabetes_binary'] = y_base

# Separate majority and minority classes
majority = data[data['Diabetes_binary'] == 0]
minority = data[data['Diabetes_binary'] == 1]

# Undersample majority class
majority_undersampled = resample(majority,
                                 replace=False,  # Sample without replacement
                                 n_samples=len(minority),  # Match the minority class size
                                 random_state=42)  # For reproducibility

# Combine minority and undersampled majority class
balanced_data = pd.concat([majority_undersampled, minority])

# Shuffle the resulting dataset
balanced_data = balanced_data.sample(frac=1, random_state=42).reset_index(drop=True)

# Split back into X and y
X = balanced_data.drop(columns=['Diabetes_binary'])
y = balanced_data['Diabetes_binary']
X_original = X.copy()
# Check the class distribution
print(y.value_counts())


## Preprocessing

### Standarization for numerical features


In [ ]:
from sklearn.preprocessing import StandardScaler

# Columns to standardize (continuous features BMI, MentHlth, and PhysHlth)
columns_to_standardize = ['BMI', 'MentHlth', 'PhysHlth']
scaler = StandardScaler()

# Scale only the selected columns
scaled_values = scaler.fit_transform(X[columns_to_standardize])  # Returns ndarray
X[columns_to_standardize] = scaled_values
print(X[columns_to_standardize].describe())


The output confirm that the standardization process was applied successfully. The mean values for each standardized featureare very close to zero, aligned with the expected outcome. The standard deviation values are approximately one for each feature, indicating that the spread of data has been adjusted to a unit scale, ensuring consistent weighting across variables. The range of values also reflects a standardized distribution, with BMI spanning from approximately -2.48 to 10.53, while MentHlth and PhysHlth show a similarly adjusted range. These results confirm that the selected features have been appropriately scaled, balancing their influence for clustering and dimensionality reduction and preparing the dataset for effective segmentation.

### Encoding categorical values

Dataset already encoded

In [284]:
# Binary, nominal, and ordinal columns
binary_columns = ['HighBP', 'HighChol', 'CholCheck','Smoker', 'Stroke', 'PhysActivity', 'Fruits', 'Veggies', 
                  'HeartDiseaseorAttack', 'HvyAlcoholConsump', 'AnyHealthcare', 'NoDocbcCost', 'DiffWalk', 'Sex']

ordinal_columns = ['Education', 'Income', 'Age', 'GenHlth'] 
numerical_columns = ['BMI', 'MentHlth', 'PhysHlth']


In [ ]:
X.head()

## Feature selection

 Variance Thresholding is used to eliminate features with low variance, as they provide limited information about the differences between data points. Correlation Analysis helps identify and remove highly correlated features, reducing redundancy without sacrificing information. 

### Variance Threshold (0.8)

In [ ]:
from sklearn.feature_selection import VarianceThreshold

# Set a threshold for variance (e.g., 0.01)
selector = VarianceThreshold(threshold=0.05)
X_var = selector.fit_transform(X)
print(f"Number of features after variance thresholding: {X_var.shape[1]}")
# Use the `get_support` method to get a boolean mask of selected features
selected_features_mask = selector.get_support()

# Get the column names corresponding to the selected features
selected_features = X.columns[selected_features_mask]
non_selected_features = X.columns[~selected_features_mask]  # Invert the mask to select non-selected features

# Print the names of the non-selected features
print("Non-selected features after variance thresholding:", non_selected_features.tolist())
# Print the names of the features selected after variance thresholding
print("Selected features after variance thresholding:", selected_features.tolist())


#### Drop low variance features

In [286]:
X= X.drop(columns=['CholCheck', 'HvyAlcoholConsump', 'AnyHealthcare'])
binary_columns = ['HighBP', 'HighChol', 'BMI', 'Smoker', 'Stroke', 'HeartDiseaseorAttack', 'PhysActivity', 'Fruits', 'Veggies', 'NoDocbcCost', 'GenHlth',
        'MentHlth', 'PhysHlth', 'DiffWalk', 'Sex', 'Age', 'Education', 'Income']

### Search for highly correlated columns (>0.8)

In [ ]:
# Compute correlation matrix
corr_matrix = X.corr()

# Find correlated features with correlation higher than a threshold of 0.8
high_corr_var = [column for column in corr_matrix.columns if any(corr_matrix[column] > 0.8)]
print("Highly correlated features:", high_corr_var)

# Drop highly correlated features
X_sel = X.drop(columns=high_corr_var)

#Plot matrix
plt.figure(figsize=(12, 10))  
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', fmt='.2f', cbar=True, linewidths=0.5)
plt.title('Correlation Matrix')
plt.show()


After applying correlation analysis to the dataset, no features were removed due to high correlation. This outcome indicates that none of the retained features exhibited correlation levels exceeding the threshold (0.8), suggesting that the remaining features are largely independent of one another. The lack of high correlation between features implies minimal redundancy, as each feature contributes unique information to the dataset.

# Algorithm implementation (Clustering)

To identify distinct patient segments within the dataset, various clustering algorithms are implemented and evaluated. Clustering is a core unsupervised machine learning technique that groups data points based on their similarity, providing insight into underlying patterns without predefined labels. Given the complexity of the dataset, which includes a range of demographic, health, and lifestyle indicators, multiple clustering methods are bieng employed to capture different aspects of the data.





In [ ]:
X.columns


In [ ]:
X.dtypes

### K-Prototypes: 2, 3 and 5 clusters

In [130]:
# Prepare data for K-Prototypes
X_np = X.to_numpy()
y = y.values

Grid search for gamma: gamma_values = np.arange(0.5, 5.5, 1)

In [ ]:
from kmodes.kprototypes import KPrototypes
from sklearn.metrics import silhouette_score, davies_bouldin_score, calinski_harabasz_score
import numpy as np


numerical_columns = ['BMI', 'MentHlth', 'PhysHlth']
categorical_columns_indices = list(set(X.columns) - set(numerical_columns))

# Convert categorical column names to indices
categorical_indices = [X.columns.get_loc(col) for col in categorical_columns_indices]

# Range of gamma values to try
gamma_values = np.arange(0.5, 5.5, 1)  # Try gamma from 0.5 to 5.0

best_gamma = None
best_score = -1
best_kproto = None

for gamma in gamma_values:

    kproto = KPrototypes(n_clusters=2, init='Huang', verbose=1, random_state=42, gamma=gamma)
    clusters = kproto.fit_predict(X_np, categorical=categorical_indices)
    
    # Compute evaluation metrics
    silhouette = silhouette_score(X_np, clusters, metric='hamming')
    dbi = davies_bouldin_score(X_np, clusters)
    chi = calinski_harabasz_score(X_np, clusters)

    # Print results for each gamma
    print(f"\nGamma: {gamma:.2f}, Clusters: 2")
    print(f"Silhouette Score: {silhouette:.3f}")
    print(f"Davies-Bouldin Index (DBI): {dbi:.3f}")
    print(f"Calinski-Harabasz Index (CHI): {chi:.3f}")

    # Track the best gamma based on silhouette score
    if silhouette > best_score:
        best_score = silhouette
        best_gamma = gamma
        best_kproto = kproto

print(f"\nBest Gamma: {best_gamma:.2f} with Silhouette Score: {best_score:.3f}")


### Categorize numerical features for ROCK and Genetic algorithms
Binning


In [288]:
X = X_original
X.drop(columns=['CholCheck', 'HvyAlcoholConsump', 'AnyHealthcare'], inplace=True)

In [289]:
# Define BMI bins and labels
bmi_bins = [0, 18.5, 25, 30, 35, 40, np.inf]
bmi_labels = [0, 1, 2, 3, 4, 5]

# Apply binning to BMI
X['BMI_Category'] = pd.cut(X['BMI'], bins=bmi_bins, labels=bmi_labels, right=False)

# Define MentHlth and PhysHlth bins and labels
health_bins = [0, 11, 21, 31]
health_labels = [0, 1, 2]

# Apply binning to MentHlth and PhysHlth
X['MentHlth_Category'] = pd.cut(X['MentHlth'], bins=health_bins, labels=health_labels, right=False)
X['PhysHlth_Category'] = pd.cut(X['PhysHlth'], bins=health_bins, labels=health_labels, right=False)

### ROCK (Robust Clustering using LinKs)

In [ ]:
X.columns

In [ ]:
import pandas as pd
from sklearn.metrics import pairwise_distances, silhouette_score, calinski_harabasz_score, davies_bouldin_score, adjusted_rand_score
from sklearn.cluster import AgglomerativeClustering

# Take a 25% random sample to avoid memory issues
X_sampled = X.sample(frac=0.25, random_state=42)  # 25% sample for avoiding memory issues
X_sampled.drop(columns=['BMI', 'MentHlth', 'PhysHlth'], inplace=True)
print(X_sampled.dtypes)

# Convert category columns to integer and drop NaN values
X_sampled['BMI_Category'] = X_sampled['BMI_Category'].astype(int)
X_sampled['MentHlth_Category'] = X_sampled['MentHlth_Category'].astype(int)
X_sampled['PhysHlth_Category'] = X_sampled['PhysHlth_Category'].astype(int)
X_sampled.dropna(inplace=True)

# Flatten true labels to ensure they are 1D
true_labels = y.loc[X_sampled.index].values.ravel()

# Define cluster values and theta grid for fine grid search
cluster_values = [2, 3, 5]
theta_values = np.arange(0.1, 1, 0.1)  #  0.1 to 0.9

# Store results 
results = []

# Grid search over theta values
for theta in theta_values:
    print(f"\n--- Evaluating for Theta = {theta:.1f} ---")
    
    # Apply thresholding to compute the pairwise Hamming distance matrix
    hamming_distance_matrix = pairwise_distances(X_sampled, metric='hamming')

    # Apply threshold to similarity 
    hamming_distance_matrix[hamming_distance_matrix > theta] = 1
    hamming_distance_matrix[hamming_distance_matrix <= theta] = 0
    
    # Ensure diagonal is zero
    np.fill_diagonal(hamming_distance_matrix, 0)

    # Loop over different cluster values
    for n_clusters in cluster_values:
        print(f"\nEvaluating with {n_clusters} clusters:")
        
        # Clustering
        clustering_sampled = AgglomerativeClustering(
            n_clusters=n_clusters, metric='precomputed', linkage='average'
        )
        clusters_sampled = clustering_sampled.fit_predict(hamming_distance_matrix)

        # Compute evaluation metrics
        sil_score = silhouette_score(hamming_distance_matrix, clusters_sampled, metric="precomputed")
        chi_score = calinski_harabasz_score(X_sampled, clusters_sampled)
        dbi_score = davies_bouldin_score(X_sampled, clusters_sampled)
        ari_score = adjusted_rand_score(true_labels, clusters_sampled)

        # Print the evaluation metrics
        print(f"Theta: {theta:.1f}, Clusters: {n_clusters}")
        print(f"Silhouette Score: {sil_score:.4f}")
        print(f"Calinski-Harabasz Index: {chi_score:.4f}")
        print(f"Davies-Bouldin Index: {dbi_score:.4f}")

        # Store results in a list for later analysis
        results.append({
            'theta': theta,
            'n_clusters': n_clusters,
            'silhouette_score': sil_score,
            'calinski_harabasz_score': chi_score,
            'davies_bouldin_score': dbi_score
        })

# Convert results to DataFrame for further analysis or visualization
results_df = pd.DataFrame(results)
print("\n--- Grid Search Results ---")
print(results_df)

### Clustering Based on Genetic Algorithms
Genetic algorithms for clustering typically involve using an evolutionary process to optimize cluster assignments. There is no direct implementation in Scikit-learn, but DEAP (Distributed Evolutionary Algorithms in Python) can be used for this.

In [ ]:
import numpy as np
from sklearn.metrics import silhouette_score, davies_bouldin_score, calinski_harabasz_score
from deap import base, creator, tools, algorithms
import random
import time
from itertools import product

# Columns info
total_columns = X.columns
categorical_indices = list(range(X.shape[1]))

# Mixed distance function for categorical data
def categorical_distance(data, centroid):
    return np.sum(data != centroid, axis=1) / data.shape[1]

# Calculate cluster assignment based on nearest centroids using Hamming distance
def assign_clusters(centroids, data):
    distances = np.array([
        categorical_distance(data, centroid)
        for centroid in centroids
    ]).T
    return np.argmin(distances, axis=1)

# minimize intra-cluster distance
def evaluate_categorical(individual, data, n_clusters):
    centroids = np.array(individual).reshape(n_clusters, -1)
    clusters = assign_clusters(centroids, data)
    intra_cluster_dist = np.sum([
        categorical_distance(np.expand_dims(data[i], axis=0), np.expand_dims(centroids[clusters[i]], axis=0))[0]
        for i in range(data.shape[0])
    ])
    penalty = 1000 if len(np.unique(clusters)) != n_clusters else 0
    return intra_cluster_dist + penalty,

# Genetic algorithm for clustering with early stopping
def genetic_clustering(data, n_clusters, n_gen=30, pop_size=20, cxpb=0.7, mutpb=0.2, patience=5):
    n_features = data.shape[1]
    
    if "FitnessMin" not in creator.__dict__:
        creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
    if "Individual" not in creator.__dict__:
        creator.create("Individual", list, fitness=creator.FitnessMin)

    toolbox = base.Toolbox()
    toolbox.register("attr_cat", random.choice, np.unique(data))
    toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_cat, n_clusters * n_features)
    toolbox.register("population", tools.initRepeat, list, toolbox.individual)

    toolbox.register("mate", tools.cxUniform, indpb=0.5)
    toolbox.register("mutate", tools.mutShuffleIndexes, indpb=0.2)
    toolbox.register("select", tools.selTournament, tournsize=3)
    toolbox.register("evaluate", evaluate_categorical, data=data, n_clusters=n_clusters)

    pop = toolbox.population(n=pop_size)
    start_time = time.time()
    evals = []

    for gen in range(n_gen):
        algorithms.eaSimple(pop, toolbox, cxpb=cxpb, mutpb=mutpb, ngen=1, verbose=False)
        best_ind = tools.selBest(pop, 1)[0]
        evals.append(best_ind.fitness.values[0])
        elapsed = time.time() - start_time
        print(f"Generation {gen+1}/{n_gen} - Best Fitness: {evals[-1]} - Elapsed Time: {elapsed:.2f}s")
        
        if len(evals) > patience and np.abs(evals[-1] - evals[-patience]) < 1e-3:
            print("Early stopping triggered.")
            break
    
    best_ind = tools.selBest(pop, 1)[0]
    best_centroids = np.array(best_ind).reshape(n_clusters, -1)

    clusters = assign_clusters(best_centroids, data)
    
    unique_clusters = np.unique(clusters)
    if len(unique_clusters) <= 1 or len(unique_clusters) >= len(data):
        silhouette, dbi, chi = -1, -1, -1
    else:
        silhouette = silhouette_score(data, clusters, metric='hamming')
        dbi = davies_bouldin_score(data, clusters)
        chi = calinski_harabasz_score(data, clusters)
    
    return clusters, best_centroids, silhouette, dbi, chi

# Hyperparameter grid search
pop_sizes = [20, 30, 50]
cxpbs = [0.6, 0.7, 0.8]

best_results = {}

for pop_size, cxpb in product(pop_sizes, cxpbs):
    print(f"Testing GA with pop_size={pop_size}, cxpb={cxpb}")
    for n_clusters in [2]:
        clusters, centroids, sil_score, dbi, chi = genetic_clustering(
            X.values, n_clusters, pop_size=pop_size, cxpb=cxpb
        )
        best_results[(n_clusters, pop_size, cxpb)] = {
            'clusters': clusters,
            'centroids': centroids,
            'silhouette_score': sil_score,
            'dbi': dbi,
            'chi': chi
        }

# Print best results
for k, v in best_results.items():
    print(f"\nClusters: {k[0]}, Pop Size: {k[1]}, Generations: {k[2]}, cxpb: {k[3]}, mutpb: {k[4]}")
    print(f"Silhouette Score: {v['silhouette_score']:.4f}")
    print(f"Davies-Bouldin Index: {v['dbi']:.4f}")
    print(f"Calinski-Harabasz Index: {v['chi']:.4f}")


# Stability analysis for best performing algorithm: ROCK


In [ ]:
import pandas as pd
from sklearn.metrics import pairwise_distances
from sklearn.cluster import AgglomerativeClustering
import numpy as np
from sklearn.model_selection import KFold
from scipy.stats import spearmanr

# parameters to study stability for each cluster configuration
best_parameters = {
    2: 0.6,  # theta for 2 clusters 
    3: 0.6,  # theta for 3 clusters
    5: 0.5   # theta for 5 clusters
}

# Initialize results list
stability_results = []

# Perform cross-validation
kf = KFold(n_splits=5, shuffle=True, random_state=42)  # 5-fold cross-validation
for n_clusters, theta in best_parameters.items():
    print(f"\nEvaluating stability for {n_clusters} clusters with theta={theta:.1f}")

    cluster_assignments = []  # To store cluster assignments for each fold
    
    for train_index, test_index in kf.split(X_sampled):
        X_train, X_test = X_sampled.iloc[train_index], X_sampled.iloc[test_index]

        # Compute Hamming distance matrix for the training data
        hamming_distance_matrix = pairwise_distances(X_train, metric='hamming')
        hamming_distance_matrix[hamming_distance_matrix > theta] = 1
        hamming_distance_matrix[hamming_distance_matrix <= theta] = 0
        np.fill_diagonal(hamming_distance_matrix, 0)
        
        # Apply Agglomerative Clustering on training data
        clustering = AgglomerativeClustering(
            n_clusters=n_clusters, metric='precomputed', linkage='average'
        )
        clusters = clustering.fit_predict(hamming_distance_matrix)
        
        # Store cluster assignments
        cluster_assignments.append(pd.Series(clusters, index=X_train.index))
    
    # Compare cluster stability across folds using Spearman's rank correlation
    pairwise_correlations = []
    for i in range(len(cluster_assignments)):
        for j in range(i + 1, len(cluster_assignments)):
            common_indices = cluster_assignments[i].index.intersection(cluster_assignments[j].index)
            if len(common_indices) > 0:
                corr, _ = spearmanr(
                    cluster_assignments[i].loc[common_indices],
                    cluster_assignments[j].loc[common_indices]
                )
                pairwise_correlations.append(corr)

    # Calculate mean stability for this cluster configuration
    mean_stability = np.mean(pairwise_correlations) if pairwise_correlations else np.nan
    print(f"Mean Stability for {n_clusters} clusters with theta={theta:.1f}: {mean_stability:.4f}")
    
    # Append to results
    stability_results.append({
        'n_clusters': n_clusters,
        'theta': theta,
        'mean_stability': mean_stability
    })

# Convert results to DataFrame for further analysis or visualization
stability_results_df = pd.DataFrame(stability_results)
print("\n--- Cluster Stability Results ---")
print(stability_results_df)


In [ ]:
# Check the cluster labels for 2 clusters
clustering = AgglomerativeClustering(n_clusters=2, metric='precomputed', linkage='average')
labels = clustering.fit_predict(hamming_distance_matrix)
print("Cluster labels for 2 clusters:", np.unique(labels))


In [ ]:
for fold, (train_idx, test_idx) in enumerate(kf.split(hamming_distance_matrix)):
    train_dist = hamming_distance_matrix[train_idx][:, train_idx]
    clustering = AgglomerativeClustering(n_clusters=2, metric='precomputed', linkage='average')
    train_labels = clustering.fit_predict(train_dist)
    print(f"Fold {fold + 1}: Cluster counts - {np.bincount(train_labels)}")


# Explicability and interpretability of the clustering
- SHAP for the global understanding of the feature importance in cluster forming. 
- LIME to obtain detailed explications (study atypic cases or miss defined clusters) 
- Finally a decision tree to generate clear rules which are visually interpretable, easing the communication of the results. 

### SHAP

In [ ]:
import shap
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# Use the final configuration for cluster assignments
optimal_theta = 0.6
optimal_clusters = 2  # best number of clusters
final_hamming_distance_matrix = pairwise_distances(X_sampled, metric='hamming')
final_hamming_distance_matrix[final_hamming_distance_matrix > optimal_theta] = 1
final_hamming_distance_matrix[final_hamming_distance_matrix <= optimal_theta] = 0
np.fill_diagonal(final_hamming_distance_matrix, 0)

final_clustering = AgglomerativeClustering(
    n_clusters=optimal_clusters, metric='precomputed', linkage='average'
)
final_clusters = final_clustering.fit_predict(final_hamming_distance_matrix)

# Train a surrogate model to predict final cluster labels
X_features = X_sampled.copy()
X_features['cluster_label'] = final_clusters  # Add cluster labels

# Define features and labels for surrogate model
X_train, X_test, y_train, y_test = train_test_split(
    X_features.drop(columns=['cluster_label']), 
    X_features['cluster_label'], 
    test_size=0.3, 
    random_state=42
)

# Train the surrogate model
surrogate_model = RandomForestClassifier(random_state=42)
surrogate_model.fit(X_train, y_train)

# Evaluate surrogate model performance
y_pred = surrogate_model.predict(X_test)
print("\n--- Surrogate Model Classification Report ---")
print(classification_report(y_test, y_pred))

# Apply SHAP to explain cluster assignments
explainer = shap.Explainer(surrogate_model, X_train)
shap_values = explainer(X_test)


In [ ]:
print("Shape of shap_values:", shap_values.shape)
print("Shape of X_test:", X_test.shape)


In [ ]:
shap_values_cluster0 = shap_values[:, :, 0]  # Select SHAP values for Cluster 0
shap.summary_plot(shap_values_cluster0, X_test)  # Summary plot for Cluster 0

shap_values_avg = shap_values.mean(axis=2)  # Average SHAP values
shap.summary_plot(shap_values_avg, X_test)  # Summary plot for averaged SHAP values


In [249]:
X_test['original_cluster_label'] = y_test

In [ ]:
print(len(shap_values_cluster0)) 

In [ ]:
print("Shape of shap_values:", shap_values.values.shape)  # Should be (n_samples, n_features)
print("Shape of X_test:", X_test.shape)  # Number of features must match

### LIME

In [ ]:
import numpy as np
import pandas as pd
from scipy.spatial.distance import cdist
from lime.lime_tabular import LimeTabularExplainer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

# Use the final configuration for cluster assignments
optimal_theta = 0.6
optimal_clusters = 2
final_hamming_distance_matrix = pairwise_distances(X_sampled, metric='hamming')
final_hamming_distance_matrix[final_hamming_distance_matrix > optimal_theta] = 1
final_hamming_distance_matrix[final_hamming_distance_matrix <= optimal_theta] = 0
np.fill_diagonal(final_hamming_distance_matrix, 0)

final_clustering = AgglomerativeClustering(
    n_clusters=optimal_clusters, metric='precomputed', linkage='average'
)
final_clusters = final_clustering.fit_predict(final_hamming_distance_matrix)

# Train-Test Split
X_features = X_sampled.copy()
X_features['cluster_label'] = final_clusters
X_train, X_test, y_train, y_test = train_test_split(
    X_features.drop(columns=['cluster_label']), 
    X_features['cluster_label'], 
    test_size=0.3, 
    random_state=42
)

# Train the surrogate model
surrogate_model = RandomForestClassifier(random_state=42)
surrogate_model.fit(X_train, y_train)

# Select representative samples
test_clusters = pd.Series(final_clusters, index=X_sampled.index).loc[X_test.index]

representative_samples = pd.concat([
    X_test[test_clusters == 0].sample(n=1, random_state=42),  # Cluster 0
    X_test[test_clusters == 1].sample(n=1, random_state=42)   # Cluster 1
])
print("\nRepresentative Samples:")
print(representative_samples)

# Select uncertain samples
probabilities = surrogate_model.predict_proba(X_test)
uncertainty_indices = np.argsort(np.abs(probabilities[:, 0] - 0.5))[:5]  # Top 5 uncertain samples
uncertain_samples = X_test.iloc[uncertainty_indices]
print("\nUncertain Samples:")
print(uncertain_samples)

# Select outliers
cluster_centers = X_test.groupby(test_clusters).mean()  # Approximate cluster centers
distances = cdist(X_test, cluster_centers, metric='hamming')
outliers = X_test.iloc[np.argmax(distances, axis=0)]  # One outlier per cluster
print("\nOutlier Samples:")
print(outliers)

# Combine all selected samples
all_selected_samples = pd.concat([representative_samples, uncertain_samples, outliers]).drop_duplicates()
print("\nAll Selected Samples for LIME:")
print(all_selected_samples)

# Apply LIME
lime_explainer = LimeTabularExplainer(
    training_data=np.array(X_train),
    feature_names=X_train.columns.tolist(),
    class_names=['Cluster 0', 'Cluster 1'],  # For binary clustering
    mode='classification'
)

for idx, sample in all_selected_samples.iterrows():
    print(f"\nExplaining sample index {idx}:\n", sample)

    # Generate LIME explanation
    lime_explanation = lime_explainer.explain_instance(
        data_row=sample.values,  # Pass the sample as a NumPy array
        predict_fn=surrogate_model.predict_proba,  # Surrogate model probabilities
        num_features=10  # Top 10 features for explanation
    )

    # Show explanation
    lime_explanation.show_in_notebook()

    # Print explanation as text
    print("\nLIME Explanation as List:")
    print(lime_explanation.as_list())


### Decision Tree

In [ ]:
# Check for NaN values in features and target
print(X_features.isnull().sum())
print(y_target.isnull().sum())


In [ ]:
print(X_sampled.shape[0], cluster_assignments[-1].shape[0])

In [ ]:
from sklearn.tree import DecisionTreeClassifier, plot_tree
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

# Select the best parameters
best_theta = 0.6
best_n_clusters = 2

# Drop rows with NaN values if any
X_sampled.dropna(inplace=True)

# Confirm no NaN values remain
if X_sampled.isnull().any().any():
    print("NaN values still present in X_sampled!")
else:
    print("No NaN values in X_sampled.")

# Recompute the best cluster labels using the best theta
hamming_distance_matrix = pairwise_distances(X_sampled, metric='hamming')
hamming_distance_matrix[hamming_distance_matrix > best_theta] = 1
hamming_distance_matrix[hamming_distance_matrix <= best_theta] = 0
np.fill_diagonal(hamming_distance_matrix, 0)

clustering_sampled = AgglomerativeClustering(
    n_clusters=best_n_clusters, metric='precomputed', linkage='average'
)
clusters_sampled = clustering_sampled.fit_predict(hamming_distance_matrix)

# Add the cluster labels to the dataset
X_sampled['Cluster_Labels'] = clusters_sampled

# Split data into train/test sets
X_features = X_sampled.drop(columns=['Cluster_Labels'])  # Features
y_clusters = X_sampled['Cluster_Labels']                # Target

X_train, X_test, y_train, y_test = train_test_split(X_features, y_clusters, test_size=0.2, random_state=42)

# Train the decision tree
decision_tree = DecisionTreeClassifier(random_state=42, max_depth=5)  # Limit depth for interpretability
decision_tree.fit(X_train, y_train)

# Evaluate the decision tree
train_score = decision_tree.score(X_train, y_train)
test_score = decision_tree.score(X_test, y_test)

print(f"Training Accuracy: {train_score:.2f}")
print(f"Test Accuracy: {test_score:.2f}")

# Visualize the decision tree
plt.figure(figsize=(20, 10))
plot_tree(decision_tree, feature_names=X_features.columns, class_names=True, filled=True)
plt.title("Decision Tree for Clustering")
plt.show()
